<a href="https://colab.research.google.com/github/tao199988/robust_cvd-main/blob/main/3DCV_Final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [3DCV Final Project] Robust Consistent Video Based on Monodepth Estimation From Single Image 

## GPU & RAM

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Dec 26 04:01:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    51W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


# Prepare the Environment

##Packages from conda
(might need clicking to restart the runtime if required)

In [4]:
%%capture
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.7.12-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local

if [ ! -f $MINICONDA_INSTALLER_SCRIPT ]; then
    wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
    chmod +x $MINICONDA_INSTALLER_SCRIPT
    ./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX
fi

conda install -y -c defaults conda python=3.6
conda update -y -c defaults --all

In [1]:
!pip install Cython==0.29.24 moviepy psutil==5.8.0 scipy==1.5.4 tensorboard==2.5.0 ipykernel==5.5.5
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install 'git+https://github.com/facebookresearch/fvcore'
!pip install -U opencv-contrib-python==3.4.2.16 opencv-python==4.5.3.56

!conda install -y -c conda-forge pybind11==2.7.0 ceres-solver==2.0.0 eigen==3.3.9 gtest==1.10.0 gflags==2.2.2 glog=0.4.0 av==8.0.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 1.9 MB/s eta 0:10:50tcmalloc: large alloc 1147494400 bytes == 0x557e425ec000 @  0x7ffbbed3f615 0x557e08954eb5 0x557e0897156f 0x557e0894fc15 0x557e089d7abc 0x557e089fa75a 0x557e089d0e66 0x557e089d1e73 0x557e089d7b95 0x557e089fa75a 0x557e089d0e66 0x557e089d1ed6 0x557e089d7b95 0x557e089fb51c 0x557e089d7eea 0x557e089faaf6 0x557e089d7eea 0x557e089faaf6 0x557e089d7eea 0x557e089faaf6 0x557e089d22db 0x557e0895001f 0x557e08954aa3 0x557e0894fa5e 0x557e089a9371 0x557e0894fe3b 0x557e089d7c0e 0x557e089fa75a 0x557e089d0e66 0x557e089d1ed6 0x557e089d7b95
     |████████████████▌               | 1055.7 MB 103.5 MB/s eta 0:00:10tcmalloc: large alloc 1434370048 bytes == 0x557e86c42000 @  0x7ffbb

## Packages from apt-get

In [2]:
%%capture
!sudo apt install \
    git \
    cmake \
    vim \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-regex-dev \
    libboost-system-dev \
    libboost-test-dev \
    libdouble-conversion-dev \
    libeigen3-dev \
    libsuitesparse-dev \
    libatlas-base-dev \
    libfreeimage-dev \
    protobuf-compiler \
    libprotobuf-dev \
    libglew-dev \
    libglvnd-dev \
    libopencv-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libcgal-qt5-dev \
    libgtk2.0-dev \

## Install Detectron2

In [3]:
%%bash
DETECTRON_REPO=/content/detectron2

if [ ! -d $DETECTRON_REPO ]; then
    python -m pip install detectron2==0.5 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html
fi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=a36251de1d6023b577be810e3cb791f99995544b49b7a0758d1c955501cfc1a7
  Stored in directory: /root/.cache/pip/wheels/db/fb/79/75c19314f02d8da5400c2b2794f5f368ab39ba7f644c98dcee
  Created wheel for pycocotools: filename=pycocotools-2.0.6-cp36-cp36m-linux_x86_64.whl size=276231 sha256=c2f59bb2d5c3cbcdfd7326dd40b15c49b1292a4e7ec0c425b0b8be02c66b9ac1
  Stored in directory: /root/.cach

## Download the robust_cvd codebase and compile

In [4]:
%%bash
sudo apt install libfmt-dev nlohmann-json-dev
cd /content
if [ ! -d /content/cvd2 ]; then
git clone https://github.com/tao199988/robust_cvd-main.git cvd2
fi
mkdir -p /content/cvd2/lib/build && cd /content/cvd2/lib/build
cmake ..
make -j16
export PYTHONPATH="${PYTHONPATH}:/content/cvd2/lib/build"

Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
Suggested packages:
  libfmt-doc
The following NEW packages will be installed:
  libfmt-dev nlohmann-json-dev
0 upgraded, 2 newly installed, 0 to remove and 20 not upgraded.
Need to get 442 kB of archives.
After this operation, 10.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 nlohmann-json-dev all 2.1.1-1.1 [364 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libfmt-dev amd64 4.0.0+ds-2 [77.6 kB]
Fetched 442 kB in 2s (222 kB/s)
Selecting previously unselected package nlohmann-json-dev.
(Reading database ... 124016 files and directories currently installed.)
Preparing to unpack .../nlohmann-json-dev_2.1.1-1.1_all.deb ...
Unpacking nlohmann-json-dev (2.1.1-1.1) ...
Selecting previously unselecte



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Cloning into 'cvd2'...


## Download models

In [5]:
%%capture
%%bash
if [ ! -d /content/cvd2/models ]; then
    cd /content/cvd2
    wget https://cloudstor.aarnet.edu.au/plus/s/lTIJF4vrvHCAI31/download -O res101.pth 
    wget https://www.dropbox.com/s/avruiwv95c5xucn/models.zip?dl=1 -O models.zip
    unzip models.zip
fi

# Prepare sample video

In [6]:
%pushd /content
!wget https://www.dropbox.com/s/vpglmx6fonaq8eu/family_run_15_frames.mov?dl=1 -O family_run.mov
%popd

/content
--2022-12-26 04:14:42--  https://www.dropbox.com/s/vpglmx6fonaq8eu/family_run_15_frames.mov?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/vpglmx6fonaq8eu/family_run_15_frames.mov [following]
--2022-12-26 04:14:42--  https://www.dropbox.com/s/dl/vpglmx6fonaq8eu/family_run_15_frames.mov
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc40a8edd8bca6d49050ccb8fd61.dl.dropboxusercontent.com/cd/0/get/BzXEekiBU5FFpARrMWf3zMXU9cjw3VycqfhhzQ_icqnW8YkiMjLfSbuXPAR505PwfHemhrf5ZUBrmug1lMV6oWzzqb1I1TBbyegwOODcJpZjYgcSW-zmRiqn-GwHy26I8fTBEqXPaLRxs4DoK_XQzhXo5fx7aGS8eWZW3Tfj7j3qBQ/file?dl=1# [following]
--2022-12-26 04:14:43--  https://uc40a8edd8bca6d49050ccb8fd61.dl.dropboxusercontent.com/cd/0/get/BzXEekiBU5FFpARrMWf3zMXU9cjw3V

# main pipeline

In [7]:
%cd /content/cvd2/

!python main.py --video_file /content/family_run.mov --path /content/Sample_output --save_intermediate_depth_streams_freq 1 --num_epochs 0 --post_filter --opt.adaptive_deformation_cost 10 --frame_range 0-15 --save_depth_visualization --load_ckpt res101.pth --backbone resnext101

/content/cvd2
['/content/cvd2', '/env/python', '/usr/local/lib/python36.zip', '/usr/local/lib/python3.6', '/usr/local/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/site-packages', '/content/cvd2/main.py', 'lib/build', '/content/cvd2/Adel_lib']
['/content/cvd2', '/env/python', '/usr/local/lib/python36.zip', '/usr/local/lib/python3.6', '/usr/local/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/site-packages', '/content/cvd2/main.py', 'lib/build', '/content/cvd2/Adel_lib', '/usr/local/lib/python3.6/site-packages/IPython/extensions', '/content/cvd2/process.py', '/content/cvd2/lib/build']
Using 1 GPUs.
C++ *NOT* logging to Stdout.
------------ Parameters -------------
align: 32
backbone: resnext101
batch_size: 2
display_freq: 100
distance_alpha: 1
distance_scale: 1
distance_type_smooth: l1
distance_type_static: l1
exp_tag: short
filter_radius: 4
flow_model: raft
flow_ops: ['hierarchical2']
frame_range: '0-15'
lambda_contrast_loss: 1.0
lambda_contrast_thresh: 1.05
lambda_dispa

# Result Visualization

In [8]:
%%capture
%cd /content/cvd2/

import os
import os.path as osp
from utils.visualization import visualize_depth_dir

# Change based on your output path.
output_dir = "Sample_output"

depth_Adelai_dir = osp.join("/content", output_dir, "depth_Adelai/depth")
depth_vis_Adelai_dir = osp.join("/content", output_dir, "depth_vis_Adelai")
os.makedirs(depth_vis_Adelai_dir, exist_ok=True)
visualize_depth_dir(depth_Adelai_dir, depth_vis_Adelai_dir)

depth_result_dir = osp.join("/content", output_dir, "R0-15_hierarchical2_Adelai/StD100.0_StR1.0_SmD0_SmR0.0/depth_e0000/e0000_filtered/depth/")
depth_vis_result_dir = osp.join("/content", output_dir,"depth_vis_result")
os.makedirs(depth_vis_result_dir, exist_ok=True)
visualize_depth_dir(depth_result_dir, depth_vis_result_dir)